In [1]:
import numpy as np
import re
from operator import itemgetter

In [69]:
Corpus={
      'l o w _':5,
       'l o w e r _':2,
       'n e w e s t _':6,
    'w i d e s t _':3,
    'h a p p i e r _':2
}

In [70]:
Corpus

{'l o w _': 5,
 'l o w e r _': 2,
 'n e w e s t _': 6,
 'w i d e s t _': 3,
 'h a p p i e r _': 2}

In [71]:
def getPairCounts(Corpus):
    pairs={}
    for word,fr in Corpus.items():
        symbols=word.split(' ')
        for i in range(len(symbols)-1):
            pair=(symbols[i],symbols[i+1])
            cfr=pairs.get(pair,0)
            pairs[pair]=cfr+fr
    return pairs

In [72]:
pairsCounts=getPairCounts(Corpus)

In [73]:
pairsCounts
    

{('l', 'o'): 7,
 ('o', 'w'): 7,
 ('w', '_'): 5,
 ('w', 'e'): 8,
 ('e', 'r'): 4,
 ('r', '_'): 4,
 ('n', 'e'): 6,
 ('e', 'w'): 6,
 ('e', 's'): 9,
 ('s', 't'): 9,
 ('t', '_'): 9,
 ('w', 'i'): 3,
 ('i', 'd'): 3,
 ('d', 'e'): 3,
 ('h', 'a'): 2,
 ('a', 'p'): 2,
 ('p', 'p'): 2,
 ('p', 'i'): 2,
 ('i', 'e'): 2}

In [90]:
def getBestPair(pairsCounts):
    return max(pairsCounts,key=pairsCounts.get)

In [91]:
getBestPair(pairsCounts)

('e', 's')

In [109]:
def mergeInCorpus(bestPair,Corpus):
    newCorpus={}
    for word in Corpus:
        newWord=re.sub(' '.join(bestPair),''.join(bestPair),word)
        newCorpus[newWord]=Corpus[word]
    return newCorpus

In [110]:
bestPair=getBestPair(pairsCounts)
t=mergeInCorpus(bestPair,Corpus)
t

{'l o w _': 5,
 'l o w e r _': 2,
 'n e w es t _': 6,
 'w i d es t _': 3,
 'h a p p i e r _': 2}

In [116]:
def runBPE(Corpus,k):
    bpeStats= {}
    for i in range(k):
        pairCounts=getPairCounts(Corpus)
        if not pairCounts:
            break
        bestPair=getBestPair(pairCounts)
        bpeStats[bestPair]=i
        Corpus=mergeInCorpus(bestPair,Corpus)
    return Corpus,bpeStats

In [117]:
Corpus={
      'l o w _':5,
       'l o w e r _':2,
       'n e w e s t _':6,
    'w i d e s t _':3,
    'h a p p i e r _':2
}

In [118]:
newCorpus,bpeStats=runBPE(Corpus,10)

In [119]:
newCorpus

{'low_': 5, 'low er _': 2, 'newest_': 6, 'w i d est_': 3, 'h a p p i er _': 2}

In [120]:
bpeStats

{('e', 's'): 0,
 ('es', 't'): 1,
 ('est', '_'): 2,
 ('l', 'o'): 3,
 ('lo', 'w'): 4,
 ('n', 'e'): 5,
 ('ne', 'w'): 6,
 ('new', 'est_'): 7,
 ('low', '_'): 8,
 ('e', 'r'): 9}

In [192]:
newword='lowest'
newword2=(' '.join(list(newword))+' _')

In [193]:
newword2

'l o w e s t _'

In [148]:
def getAllPairs(word):
    pairs=[]
    word=word.split(' ')
    prevChar=word[0]
    for char in word[1:]:
        pairs.append((prevChar,char))
        prevChar=char
    return pairs

In [149]:
pairs=getAllPairs(newword2)

In [150]:
pairs

[('l', 'o'), ('o', 'w'), ('w', 'e'), ('e', 's'), ('s', 't_')]

In [176]:
def getPairToBeMerged(bpeStats,pairs):
    bpeCodes=[(pair,bpeStats[pair]) for pair in pairs if pair in bpeStats]
    if len(bpeCodes)==0:
        return (-1,-1)
    pairToBeMerged=min(bpeCodes,key=itemgetter(1))[0]
    return pairToBeMerged
    

In [177]:
pairToBeMerged=getPairToBeMerged(bpeStats,pairs)

In [169]:
def mergeLetters(word,pairToBeMerged):
    newword=re.sub(' '.join(pairToBeMerged),''.join(pairToBeMerged),word)
    return newword

In [170]:
mergeLetters(newword2,pairToBeMerged)

'l o w es t_'

In [200]:
def bpeTokenize(word,bpeStats):
    if len(word)== 1:
        return word
    word=' '.join(list(word))+' _'
    while True:
        pairs=getAllPairs(word)
        pairToBeMerged= getPairToBeMerged(bpeStats,pairs)
        if pairToBeMerged[0] == -1:
            break
        word= mergeLetters(word,pairToBeMerged)
    return word
    

In [201]:
newWord=bpeTokenize('slowest',bpeStats)

In [202]:
newWord

's low est_'